In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import csv

In [23]:
# load vsech nasich + Honzovo dat o pozicich autobusu a spojeni do velke tabulky, odstaneni radku, kde je delay = NaN
data_Honza = pd.read_csv('out.c-cleaning.ALL_HONZA_NEW.csv', encoding='utf-8', low_memory=False)
data_Honza_wkend = pd.read_csv('out.c-cleaning.ALL_HONZA_NEW_WKEND.csv', encoding='utf-8', low_memory=False)
data_keboola = pd.read_csv('out.c-cleaning.ALL_POSITIONS_NEW.csv', encoding='utf-8', low_memory=False)
data_keboola_wkend = pd.read_csv('out_clean_vikend_16_17_23_24.csv', encoding='utf-8', low_memory=False)

data = pd.concat([data_Honza,data_Honza_wkend,data_keboola,data_keboola_wkend], ignore_index = True)
data = data.dropna(axis=0,subset=['DELAY'],how='any')
data.head()

,UNICORN,TRIP_ID,START_TIME,HEADSIGN,DATE,BEARING,DELAY,DELAY_STOP_ARRIVAL,DELAY_STOP_DEPARTURE,IS_CANCELED,...,LAST_STOP_SEQUENCE,NEXT_STOP_ARRIVAL_TIME,NEXT_STOP_DEPARTURE_TIME,NEXT_STOP_ID,NEXT_STOP_SEQUENCE,ORIGIN_TIMESTAMP,SHAPE_DIST_TRAVELED,SPEED,LATITUDE,LONGITUDE
0,331_262_200302_06_May,331_262_200302,17:36:00,"Jílové u Prahy,Náměstí",06 May,197,-79.0,-98.0,NaN,FALSE,...,27.0,"Wed, 06 May 2020 18:27:00 Z","Wed, 06 May 2020 18:27:00 Z",U2109Z3,28.0,"Wed, 06 May 2020 18:25:35 Z",22.7,0,49.89513,14.49307
1,331_262_200302_06_May,331_262_200302,17:36:00,"Jílové u Prahy,Náměstí",06 May,100,-100.0,NaN,-59.0,FALSE,...,27.0,"Wed, 06 May 2020 18:27:00 Z","Wed, 06 May 2020 18:27:00 Z",U2109Z3,28.0,"Wed, 06 May 2020 18:24:57 Z",22.6,29,49.89572,14.49165
2,331_262_200302_06_May,331_262_200302,17:36:00,"Jílové u Prahy,Náměstí",06 May,126,-71.0,NaN,-59.0,FALSE,...,27.0,"Wed, 06 May 2020 18:27:00 Z","Wed, 06 May 2020 18:27:00 Z",U2109Z3,28.0,"Wed, 06 May 2020 18:24:35 Z",22.3,40,49.89653,14.48904
3,331_262_200302_06_May,331_262_200302,17:36:00,"Jílové u Prahy,Náměstí",06 May,108,-63.0,NaN,-59.0,FALSE,...,27.0,"Wed, 06 May 2020 18:27:00 Z","Wed, 06 May 2020 18:27:00 Z",U2109Z3,28.0,"Wed, 06 May 2020 18:24:10 Z",22.1,33,49.89730,14.48636
4,331_262_200302_06_May,331_262_200302,17:36:00,"Jílové u Prahy,Náměstí",06 May,64,-46.0,NaN,29.0,FALSE,...,26.0,"Wed, 06 May 2020 18:25:00 Z","Wed, 06 May 2020 18:25:00 Z",U2100Z1,27.0,"Wed, 06 May 2020 18:23:41 Z",21.8,41,49.89767,14.48212


In [24]:
# pridani sloupce s route_name
data['ROUTE_NAME'] = data['TRIP_ID'].str[:3]

In [25]:
# prevod sloupecku DATE na datovy format
data['DATE'] = data['DATE'] + " 2020"

In [26]:
data['DATE'] = pd.to_datetime(data['DATE'])

In [27]:
# vytvoreni noveho sloupce se dnem v tydnu
data['DAY_OF_WEEK'] = data['DATE'].dt.day_name()

In [28]:
data['DAY_OF_WEEK_NUM'] = data['DATE'].dt.dayofweek

In [37]:
# filtrace pouze linky 331 a 333, v ranni spicce, smer centrum -> novy mensi dataframe
morn_331_333 = data.loc[(data['HEADSIGN'] == 'Praha,Kačerov') 
                     & (data['START_TIME'] < '12:00:00') 
                     & ((data['ROUTE_NAME'] == '333') | (data['ROUTE_NAME'] == '331'))
                     , ['UNICORN', 'TRIP_ID', 'START_TIME', 'HEADSIGN', 'DATE',
                        'DELAY', 'DELAY_STOP_ARRIVAL', 'DELAY_STOP_DEPARTURE',
                        'ORIGIN_TIMESTAMP', 'SHAPE_DIST_TRAVELED', 'SPEED', 'LATITUDE',
                        'LONGITUDE','ROUTE_NAME','DAY_OF_WEEK']]

#morn_331_333.head()

In [38]:
# pocet unikatnich trip_id v souboru 
len(morn_331_333['TRIP_ID'].unique())

27

In [39]:
# DataFrame se vsemi jizdnimi rady (pro unikatnich 27 trip_id) -> vytvoreno v jizdni_rad.ipynb
info_stops_clean = pd.read_csv('jizdnirad.csv', encoding='utf-8', low_memory=False)

In [40]:
info_stops_clean.head()

,ARRIVAL_TIME,DEPARTURE_TIME,SHAPE_DIST_TRAVELED,STOP_ID,STOP_SEQUENCE,TRIP_ID,GEOMETRY_COORDINATES,STOP_NAME
0,9:31:00,9:31:00,0.00000,U2109Z2,1,331_243_200302,"[14.49348, 49.89506]","Jílové u Prahy,Náměstí"
1,9:33:00,9:33:00,0.65952,U2100Z2,2,331_243_200302,"[14.48552, 49.8974]","Jílové u Prahy,Učiliště"
2,9:37:00,9:37:00,2.37567,U1586Z2,3,331_243_200302,"[14.46857, 49.89977]","Jílové u Prahy,Kamenná Vrata"
3,9:39:00,9:39:00,4.03046,U1521Z2,4,331_243_200302,"[14.45916, 49.90944]","Okrouhlo,Zahořany"
4,9:43:00,9:43:00,5.86913,U1520Z2,5,331_243_200302,"[14.44766, 49.91885]",Okrouhlo


In [41]:
# vybrani sloupcu z JR
stop_dist = info_stops_clean.loc[:,['TRIP_ID','SHAPE_DIST_TRAVELED','STOP_ID','STOP_SEQUENCE','STOP_NAME']]
# stop_dist

In [42]:
# list unikatnich unicornu z df data (allpositions)
unicorn_list = morn_331_333['UNICORN'].unique().tolist()
len(unicorn_list)

335

In [43]:
# prazdny novy DataFrame
final_tab = pd.DataFrame([], 
                         columns = ['DAY_OF_WEEK','TRIP_ID','DATE','UNICORN','STOP_ID','STOP_SEQUENCE','SHAPE_DIST_TRAVELED','DELAY'])
final_tab

,DAY_OF_WEEK,TRIP_ID,DATE,UNICORN,STOP_ID,STOP_SEQUENCE,SHAPE_DIST_TRAVELED,DELAY,DEPARTURE_TIME


In [47]:
index = 0
final_tab = pd.DataFrame()

for uni in unicorn_list:
    #tohle je jen pro vypisování stavu procesu
    print(round(index/len(unicorn_list)*100), '%')
    index+=1
    
    #vyberu vsechny pozice od daného spoje
    unicorn_find = morn_331_333[morn_331_333['UNICORN'] == uni]
    
    #vyberu jen sloupce co mě zajímají
    bus_dist = unicorn_find.loc[:,['TRIP_ID','SHAPE_DIST_TRAVELED','DELAY','DATE','DAY_OF_WEEK',
                                   'START_TIME','HEADSIGN','ROUTE_NAME','ORIGIN_TIMESTAMP']]
    
    #tady si vytáhnu opakující se trip_id i trip_date
    gtfs_trip_id = bus_dist['TRIP_ID'].tolist()[0]
    trip_date = bus_dist['DATE'].tolist()[0]
    trip_day = bus_dist['DAY_OF_WEEK'].tolist()[0]
    trip_start_time = bus_dist['START_TIME'].tolist()[0]
    trip_headsign = bus_dist['HEADSIGN'].tolist()[0]
    route_name = bus_dist['ROUTE_NAME'].tolist()[0]
    origin_timestamp = bus_dist['ORIGIN_TIMESTAMP'].tolist()[0]
    
    
    #pro každou zastávku v JŘ pro dané GTFS_TRIP_ID
    for i, stop in stop_dist[stop_dist['TRIP_ID'] == gtfs_trip_id].iterrows():
        
        #hledám nejbližší pozici tím, že si nastavím zatím nejbližší pozici jako hooodně daleko
        min_distance_so_far = 100000
        #i to zpoždění na nulu, ale prostě se vybere určitě alespoň jedna reálná
        stop_delay = 0
        
        for j, position in bus_dist.iterrows():
            
            # vypocita absolutni hodnotu rozdilu vzdalenosti
            actual_distance = abs(stop['SHAPE_DIST_TRAVELED'] - position['SHAPE_DIST_TRAVELED'])

            # a pokud je tahle pozice blíž než 100000 nebo blíž než poslední jiná nalezená
            if (min_distance_so_far > actual_distance):
                #tak si uližím aktuální nejbližší vzdálenost
                min_distance_so_far = actual_distance
                #a uložím si k ní i její zpoždění
                stop_delay = position['DELAY']
                origin_timestamp = position['ORIGIN_TIMESTAMP']
            

        # teď už mi to projelo k dané zastávce všema pozicema a snad jsem nalezl opravdu tu nejbližší
        #udělám si výsledný dataframe se všemi potřebnými údaji
        df = pd.DataFrame(
            {
                "TRIP_ID": [gtfs_trip_id],
                "ROUTE" : [route_name],
                "DAY_OF_WEEK": [trip_day],
                "DATE": [trip_date],
                "UNICORN": [uni],
                "START_TIME" : [trip_start_time],
                "HEADSIGN" : [trip_headsign],
                "STOP_ID": [stop['STOP_ID']],
                "STOP_SEQUENCE": [stop['STOP_SEQUENCE']],
                "STOP_NAME" : [stop['STOP_NAME']],
                "SHAPE_DIST_TRAVELED": [stop['SHAPE_DIST_TRAVELED']],
                "DELAY": [stop_delay],
                "ORIGIN_TIMESTAMP": [origin_timestamp],
                
                
                
            }
        )
        # a uložím do jednoho velkého
        final_tab = final_tab.append(df, ignore_index = True)

0 %
0 %
1 %
1 %
1 %
1 %
2 %
2 %
2 %
3 %
3 %
3 %
4 %
4 %
4 %
4 %
5 %
5 %
5 %
6 %
6 %
6 %
7 %
7 %
7 %
7 %
8 %
8 %
8 %
9 %
9 %
9 %
10 %
10 %
10 %
10 %
11 %
11 %
11 %
12 %
12 %
12 %
13 %
13 %
13 %
13 %
14 %
14 %
14 %
15 %
15 %
15 %
16 %
16 %
16 %
16 %
17 %
17 %
17 %
18 %
18 %
18 %
19 %
19 %
19 %
19 %
20 %
20 %
20 %
21 %
21 %
21 %
21 %
22 %
22 %
22 %
23 %
23 %
23 %
24 %
24 %
24 %
24 %
25 %
25 %
25 %
26 %
26 %
26 %
27 %
27 %
27 %
27 %
28 %
28 %
28 %
29 %
29 %
29 %
30 %
30 %
30 %
30 %
31 %
31 %
31 %
32 %
32 %
32 %
33 %
33 %
33 %
33 %
34 %
34 %
34 %
35 %
35 %
35 %
36 %
36 %
36 %
36 %
37 %
37 %
37 %
38 %
38 %
38 %
39 %
39 %
39 %
39 %
40 %
40 %
40 %
41 %
41 %
41 %
41 %
42 %
42 %
42 %
43 %
43 %
43 %
44 %
44 %
44 %
44 %
45 %
45 %
45 %
46 %
46 %
46 %
47 %
47 %
47 %
47 %
48 %
48 %
48 %
49 %
49 %
49 %
50 %
50 %
50 %
50 %
51 %
51 %
51 %
52 %
52 %
52 %
53 %
53 %
53 %
53 %
54 %
54 %
54 %
55 %
55 %
55 %
56 %
56 %
56 %
56 %
57 %
57 %
57 %
58 %
58 %
58 %
59 %
59 %
59 %
59 %
60 %
60 %
60 %
61 %
61 %
61 %
61

In [48]:
#tohle jsou všechny trip_id a podíváme se
len(final_tab['UNICORN'].unique())

335

In [49]:
final_tab

,TRIP_ID,ROUTE,DAY_OF_WEEK,DATE,UNICORN,START_TIME,HEADSIGN,STOP_ID,STOP_SEQUENCE,STOP_NAME,SHAPE_DIST_TRAVELED,DELAY,ORIGIN_TIMESTAMP
0,331_241_200302,331,Wednesday,2020-05-06,331_241_200302_06_May,06:46:00,"Praha,Kačerov",U2109Z2,1,"Jílové u Prahy,Náměstí",0.00000,8.0,"Wed, 06 May 2020 06:46:08 Z"
1,331_241_200302,331,Wednesday,2020-05-06,331_241_200302_06_May,06:46:00,"Praha,Kačerov",U2100Z2,2,"Jílové u Prahy,Učiliště",0.65952,-10.0,"Wed, 06 May 2020 06:47:56 Z"
2,331_241_200302,331,Wednesday,2020-05-06,331_241_200302_06_May,06:46:00,"Praha,Kačerov",U1586Z2,3,"Jílové u Prahy,Kamenná Vrata",2.37567,-68.0,"Wed, 06 May 2020 06:50:54 Z"
3,331_241_200302,331,Wednesday,2020-05-06,331_241_200302_06_May,06:46:00,"Praha,Kačerov",U1521Z2,4,"Okrouhlo,Zahořany",4.03046,14.0,"Wed, 06 May 2020 06:54:12 Z"
4,331_241_200302,331,Wednesday,2020-05-06,331_241_200302_06_May,06:46:00,"Praha,Kačerov",U1520Z2,5,Okrouhlo,5.86913,9.0,"Wed, 06 May 2020 06:58:11 Z"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7861,333_751_200302,333,Tuesday,2020-05-19,333_751_200302_19_May,08:48:00,"Praha,Kačerov",U749Z2,12,Tempo,6.99949,29.0,"Tue, 19 May 2020 09:04:29 Z"
7862,333_751_200302,333,Tuesday,2020-05-19,333_751_200302_19_May,08:48:00,"Praha,Kačerov",U656Z2,13,Sídliště Krč,7.33832,14.0,"Tue, 19 May 2020 09:05:07 Z"
7863,333_751_200302,333,Tuesday,2020-05-19,333_751_200302_19_May,08:48:00,"Praha,Kačerov",U893Z4,14,Zálesí,7.80231,26.0,"Tue, 19 May 2020 09:06:26 Z"
7864,333_751_200302,333,Tuesday,2020-05-19,333_751_200302_19_May,08:48:00,"Praha,Kačerov",U488Z7,15,Nemocnice Krč,8.35517,25.0,"Tue, 19 May 2020 09:08:33 Z"


In [50]:
final_tab['TRIP_ID'].unique()

array(['331_241_200302', '333_557_200302', '331_243_200302',
       '333_1064_200302', '333_715_200302', '333_1072_200302',
       '333_1052_200302', '331_260_200302', '333_677_200302',
       '333_751_200302', '333_1012_200302', '333_1094_200302',
       '333_1115_200302', '331_257_200302', '333_651_200307',
       '333_1026_200307', '333_653_200307', '333_671_200307',
       '333_1010_200307', '333_1037_200307', '333_636_200307',
       '333_1014_200307', '333_1047_200307', '333_655_200307',
       '333_1024_200307', '333_673_200307', '333_1012_200307'],
      dtype=object)

Unique trip_id --> sjednocení STOP_SEQUENCE ze zastávky Dolní Břežany,Náměstí

333_655_200307 >= 2
333_1012_200307 >= 14 
333_655_200307 Dolní Břežany,Náměstí
333_1047_200307(Zvole) >= 7 
333_636_200307 Dolní Břežany,Náměstí
333_1037_200307(Březová-Oleško,Oleško) >= 14 
333_1010_200307(Březová-Oleško,Oleško) >= 19   
333_653_200307 Dolní Břežany,Náměstí	
333_1026_200307(Zvole) >= 14
333_651_200307 Dolní Břežany,Náměstí	
333_1115_200302 (Zvole) >=12
333_1094_200302	(Březová-Oleško,Oleško) >= 14
333_1012_200302(Březová-Oleško,Oleško) >= 14
333_751_200302 (Dolní Břežany,Obecní úřad) >= 2
333_677_200302 (Dolní Břežany,Obecní úřad) >= 2
333_1052_200302(Zvole) >= 12
333_1072_200302(Zvole,Nová Zvole) >= 14
333_715_200302 (Dolní Břežany,Obecní úřad) >= 2
333_1064_200302 (Březová-Oleško,Oleško) >= 14
333_557_200302 (Dolní Břežany,Zálepy) >= 7
333_1014_200307 (Zvole) >= 14
333_671_200307 Dolní Břežany,Náměstí
333_1024_200307 (Zvole) >= 7
333_673_200307 Dolní Břežany,Náměstí

331_241_200302 (Jílové u Prahy,Náměstí) >= 14	
331_243_200302 (Jílové u Prahy,Náměstí) >= 14	
331_257_200302 (Okrouhlo,Zahořany) >= 11
331_260_200302 (Zvole) >= 8

In [51]:
# vyradit podle sequence stop prvni zastavky nezajimave pro  nas
final_clean =   final_tab[
((final_tab['TRIP_ID'] == '333_677_200302') & (final_tab['STOP_SEQUENCE'] >= 2)) 
| ((final_tab['TRIP_ID'] == '333_715_200302') & (final_tab['STOP_SEQUENCE'] >= 2))
| ((final_tab['TRIP_ID'] == '333_751_200302') & (final_tab['STOP_SEQUENCE'] >= 2))
| ((final_tab['TRIP_ID'] == '333_1012_200307') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_1047_200307') & (final_tab['STOP_SEQUENCE'] >= 7))
| ((final_tab['TRIP_ID'] == '333_1037_200307') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_1010_200307') & (final_tab['STOP_SEQUENCE'] >= 19))
| ((final_tab['TRIP_ID'] == '333_1026_200307') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_1115_200302') & (final_tab['STOP_SEQUENCE'] >= 12))
| ((final_tab['TRIP_ID'] == '333_1094_200302') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_1012_200302') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_1052_200302') & (final_tab['STOP_SEQUENCE'] >= 12))
| ((final_tab['TRIP_ID'] == '333_1072_200302') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_1064_200302') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_557_200302') & (final_tab['STOP_SEQUENCE'] >= 7))
| ((final_tab['TRIP_ID'] == '331_241_200302') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '331_243_200302') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '331_257_200302') & (final_tab['STOP_SEQUENCE'] >= 11))
| ((final_tab['TRIP_ID'] == '331_260_200302') & (final_tab['STOP_SEQUENCE'] >= 8))
| (final_tab['TRIP_ID'] == '333_636_200307')
| (final_tab['TRIP_ID'] == '333_655_200307')
| (final_tab['TRIP_ID'] == '333_653_200307')
| (final_tab['TRIP_ID'] == '333_651_200307')
| (final_tab['TRIP_ID'] == '333_671_200307')
| ((final_tab['TRIP_ID'] == '333_1014_200307') & (final_tab['STOP_SEQUENCE'] >= 14))
| ((final_tab['TRIP_ID'] == '333_1024_200307') & (final_tab['STOP_SEQUENCE'] >= 7))
| (final_tab['TRIP_ID'] == '333_673_200307')
]

final_clean

,TRIP_ID,ROUTE,DAY_OF_WEEK,DATE,UNICORN,START_TIME,HEADSIGN,STOP_ID,STOP_SEQUENCE,STOP_NAME,SHAPE_DIST_TRAVELED,DELAY,ORIGIN_TIMESTAMP
13,331_241_200302,331,Wednesday,2020-05-06,331_241_200302_06_May,06:46:00,"Praha,Kačerov",U1502Z2,14,"Dolní Břežany,Náměstí",13.17849,98.0,"Wed, 06 May 2020 07:16:42 Z"
14,331_241_200302,331,Wednesday,2020-05-06,331_241_200302_06_May,06:46:00,"Praha,Kačerov",U1501Z2,15,"Dolní Břežany,Na kopečku",13.80006,32.0,"Wed, 06 May 2020 07:17:32 Z"
15,331_241_200302,331,Wednesday,2020-05-06,331_241_200302_06_May,06:46:00,"Praha,Kačerov",U545Z2,16,Písnice,16.55734,45.0,"Wed, 06 May 2020 07:21:40 Z"
16,331_241_200302,331,Wednesday,2020-05-06,331_241_200302_06_May,06:46:00,"Praha,Kačerov",U546Z2,17,Ke Březině,16.93703,29.0,"Wed, 06 May 2020 07:22:48 Z"
17,331_241_200302,331,Wednesday,2020-05-06,331_241_200302_06_May,06:46:00,"Praha,Kačerov",U871Z2,18,Lipovická,17.13697,11.0,"Wed, 06 May 2020 07:23:19 Z"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7861,333_751_200302,333,Tuesday,2020-05-19,333_751_200302_19_May,08:48:00,"Praha,Kačerov",U749Z2,12,Tempo,6.99949,29.0,"Tue, 19 May 2020 09:04:29 Z"
7862,333_751_200302,333,Tuesday,2020-05-19,333_751_200302_19_May,08:48:00,"Praha,Kačerov",U656Z2,13,Sídliště Krč,7.33832,14.0,"Tue, 19 May 2020 09:05:07 Z"
7863,333_751_200302,333,Tuesday,2020-05-19,333_751_200302_19_May,08:48:00,"Praha,Kačerov",U893Z4,14,Zálesí,7.80231,26.0,"Tue, 19 May 2020 09:06:26 Z"
7864,333_751_200302,333,Tuesday,2020-05-19,333_751_200302_19_May,08:48:00,"Praha,Kačerov",U488Z7,15,Nemocnice Krč,8.35517,25.0,"Tue, 19 May 2020 09:08:33 Z"


In [52]:
#zkousim si jestli jsem usekla správný úsek
#final_clean[(final_clean['STOP_NAME'] == 'Dolní Břežany,Náměstí')]
len(final_clean.loc[:,'UNICORN'].unique())

335

In [53]:
final_clean.to_csv('stops_vs_positions_clean.csv', encoding="utf-8", index=False)